# Connect to PostgreSQL from Jupyter (R)

This notebook uses the R kernel to connect to a PostgreSQL database using environment variables stored in a `.env` file. It is beginner-friendly and ready to run on macOS when Jupyter is running on the same machine as PostgreSQL.

What this notebook does:
- Shows a safe way to load a `.env` file (without printing secrets)
- Connects to PostgreSQL using DBI + RPostgres
- Runs a sample query and shows the results

Notes before running:
- Create a `.env` file next to this notebook (example below).
- Make sure the R kernel (IRkernel) is installed and selected.
- On macOS you may need to have libpq available (Homebrew `postgresql` provides it).


## Example .env file

Create a file named `.env` in the same folder as this notebook with these contents (replace values):

```
PGHOST=localhost
PGPORT=5432
PGUSER=myuser
PGPASSWORD=mysecretpassword
PGDATABASE=mydb
```

Do NOT commit `.env` to version control. Add `.env` to your `.gitignore`.


In [2]:
# Install required packages if they are missing (runs in R)
if (!requireNamespace("DBI", quietly = TRUE)) install.packages("DBI", repos = "https://cloud.r-project.org")
if (!requireNamespace("RPostgres", quietly = TRUE)) install.packages("RPostgres", repos = "https://cloud.r-project.org")
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr", repos = "https://cloud.r-project.org")
# dotenv is optional; we'll fallback to a small loader if it's not available
if (!requireNamespace("dotenv", quietly = TRUE)) {
  message("Package 'dotenv' not installed; falling back to a built-in .env loader (that's fine).")
} else {
  message("Package 'dotenv' is available; we'll prefer it to load .env if present.")
}


Package 'dotenv' not installed; falling back to a built-in .env loader (that's fine).



In [3]:
# Load libraries
library(DBI)
library(RPostgres)
library(dplyr)

# Corrected .env parser and loader
load_env_file <- function(file = ".env") {
  if (!file.exists(file)) {
    message("No .env file found at: ", file)
    return(invisible(FALSE))
  }

  lines <- readLines(file, warn = FALSE)
  lines <- trimws(lines)
  lines <- lines[lines != "" & !grepl("^\\s*#", lines)]  # ignore blank and comment lines

  for (line in lines) {
    parts <- strsplit(line, "=", fixed = TRUE)[[1]]
    key <- trimws(parts[1])
    val <- if (length(parts) > 1) paste(parts[-1], collapse = "=") else ""

    # trim whitespace around the value
    val <- trimws(val)

    # remove surrounding single or double quotes if present
    # pattern uses double-quoted string with escaped double quote and single quote inside
    val <- gsub("^['\\\"]|['\\\"]$", "", val)

    # set the environment variable
    do.call(Sys.setenv, setNames(list(val), key))
  }

  invisible(TRUE)
}

# Try to load .env: prefer dotenv package if available, otherwise use the helper above
env_loaded <- FALSE
if (requireNamespace("dotenv", quietly = TRUE)) {
  try({
    dotenv::load_dot_env()
    env_loaded <- TRUE
  }, silent = TRUE)
}
if (!env_loaded) {
  load_env_file(".env")
}

# Show confirmation (do NOT print secrets)
cat("PGHOST:", Sys.getenv("PGHOST", unset = "<not set>"), "\n")
cat("PGPORT:", Sys.getenv("PGPORT", unset = "<not set>"), "\n")
cat("PGDATABASE:", Sys.getenv("PGDATABASE", unset = "<not set>"), "\n")
cat("PGUSER:", Sys.getenv("PGUSER", unset = "<not set>"), "\n")
if (nzchar(Sys.getenv("PGPASSWORD"))) cat("PGPASSWORD: (loaded, not printed)\n") else cat("PGPASSWORD: (not set)\n")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




PGHOST: localhost 
PGPORT: 5435 
PGDATABASE: erindamm 
PGUSER: erindamm 
PGPASSWORD: (loaded, not printed)


In [4]:
# Basic validation: stop with a helpful message if required vars are missing
required <- c("PGUSER", "PGPASSWORD", "PGDATABASE")
missing <- required[!nzchar(Sys.getenv(required))]
if (length(missing) > 0) {
  stop("Missing required environment variables: ", paste(missing, collapse = ", "),
       "\n\nPlease create a .env file with these values next to this notebook, or set the env vars in your shell.")
}


In [5]:
# Run this whole cell
library(DBI)
library(RPostgres)

# Helper: get a plain string (works whether you have a named vector or not)
get_str <- function(x, default = "") {
  # if x is a named 1-element vector, extract the value; otherwise coerce to scalar string
  if (is.null(x)) return(default)
  if (length(x) == 0) return(default)
  # if it's a named vector like env_vals["PGDATABASE"], use [[ or unname
  val <- if (is.vector(x) && !is.null(names(x)) && names(x)[1] != "") {
    unname(x)[1]
  } else if (is.list(x) && length(x) >= 1) {
    as.character(x[[1]])
  } else {
    as.character(x[1])
  }
  if (is.na(val) || !nzchar(val)) return(default)
  val
}

# Try reading straight from Sys.getenv (preferred)
pg_host <- get_str(Sys.getenv("PGHOST", unset = "localhost"))
pg_port <- as.integer(get_str(Sys.getenv("PGPORT", unset = "5432")))
pg_user <- get_str(Sys.getenv("PGUSER", unset = ""))
pg_password <- get_str(Sys.getenv("PGPASSWORD", unset = ""))
pg_db <- get_str(Sys.getenv("PGDATABASE", unset = ""))

# If you used an env_vals vector earlier (e.g. env_vals <- c(...); pg_db <- env_vals["PGDATABASE"]),
# the code above still handles it. But if you still have env_vals in your session, prefer:
# pg_db <- get_str(if (exists("env_vals")) env_vals["PGDATABASE"] else Sys.getenv("PGDATABASE"))

cat("Using:\n")
cat("  host:", pg_host, "\n")
cat("  port:", pg_port, "\n")
cat("  user:", if (nzchar(pg_user)) pg_user else "<not set>", "\n")
cat("  database:", if (nzchar(pg_db)) pg_db else "<not set>", "\n")
cat("  password loaded?:", nzchar(pg_password), "\n\n")

# Validate
missing <- character()
if (!nzchar(pg_user)) missing <- c(missing, "PGUSER")
if (!nzchar(pg_password)) missing <- c(missing, "PGPASSWORD")
if (!nzchar(pg_db)) missing <- c(missing, "PGDATABASE")
if (length(missing) > 0) {
  stop("Missing required env vars: ", paste(missing, collapse = ", "),
       "\nSet them (e.g. Sys.setenv(PGDATABASE='mydb')) or re-run your .env loader cell, then run this cell again.")
}

Using:
  host: localhost 
  port: 5435 
  user: erindamm 
  database: erindamm 
  password loaded?: TRUE 



In [6]:
# Connect to PostgreSQL
con <- dbConnect(
  RPostgres::Postgres(),
  host = pg_host,
  port = pg_port,
  dbname = pg_db,
  user = pg_user,
  password = pg_password
)

cat("Connection established.\n")

# List tables (shows first few if many)
tables <- dbListTables(con)
cat("Tables (first 20):\n")
print(head(tables, 20))


Connection established.
Tables (first 20):
[1] "survey_605"


In [8]:
# Example: read a small sample from a table named 'my_table'
# Replace 'my_table' with a real table name in your database.
sample_table_name <- "survey_605"  # <-- change this to your table

if (sample_table_name %in% tables) {
  df <- dbGetQuery(con, sprintf("SELECT * FROM %s LIMIT 10;", DBI::SQL(sample_table_name)))
  print(df)
} else {
  cat("Table '", sample_table_name, "' not found. Pick a table from the list above.\n", sep = "")
}


    response_timestamp      age gender                  education_level
1  2023-03-31 03:13:19    18-24   Male Some college or associate degree
2  2023-03-31 22:37:46 Under 18   Male                Bachelor's degree
3  2023-04-01 01:14:46    18-24 Female                Bachelor's degree
4  2023-04-01 03:06:07    25-34 Female Some college or associate degree
5  2023-04-01 03:07:32    18-24   Male                Bachelor's degree
6  2023-04-01 03:08:56    18-24 Female                  Master's degree
7  2023-04-01 04:40:50    18-24   Male                Bachelor's degree
8  2023-04-02 17:20:08    18-24 Female                Bachelor's degree
9  2023-04-02 17:23:14    18-24   Male              High school diploma
10 2023-04-02 17:23:50    35-44   Male              High school diploma
           occupation weekly_exercise_frequency length_wearable_history
1             Student    5 or more times a week      Less than 6 months
2             Student    5 or more times a week      Less than 6

In [9]:
# You can also use dplyr syntax connected to the database
if (sample_table_name %in% tables) {
  tbl(con, sample_table_name) %>%
    head(10) %>%
    collect() %>%
    print()
} else {
  cat("Skipping dplyr example because table not found.\n")
}

# A tibble: 10 × 22
   response_timestamp  age      gender education_level                occupation
   <dttm>              <chr>    <chr>  <chr>                          <chr>     
 1 2023-03-31 03:13:19 18-24    Male   Some college or associate deg… Student   
 2 2023-03-31 22:37:46 Under 18 Male   Bachelor's degree              Student   
 3 2023-04-01 01:14:46 18-24    Female Bachelor's degree              Student   
 4 2023-04-01 03:06:07 25-34    Female Some college or associate deg… Employed …
 5 2023-04-01 03:07:32 18-24    Male   Bachelor's degree              Student   
 6 2023-04-01 03:08:56 18-24    Female Master's degree                Employed …
 7 2023-04-01 04:40:50 18-24    Male   Bachelor's degree              Student   
 8 2023-04-02 17:20:08 18-24    Female Bachelor's degree              Student   
 9 2023-04-02 17:23:14 18-24    Male   High school diploma            Employed …
10 2023-04-02 17:23:50 35-44    Male   High school diploma            Employed …
# ℹ 17 m

In [9]:
# Clean up: disconnect when you're done
dbDisconnect(con)
cat("Disconnected.\n")


Disconnected.
